1.Create Benchmark Instance

In [1]:
from nats_bench import create
from pprint import pprint

# Create the API instance for the size search space in NATS
api = create('/work/ws-tmp/g051463-tenneti_NAS/Thesis/data/NATS-tss-v1_0-3ffb9-full/', 'tss', fast_mode=True, verbose=True)
print('\nAPI create done: {:}\n'.format(api))


info = api.get_more_info(1200, 'cifar100', hp='200')
pprint(info)

[2023-01-25 22:35:47] Try to create the NATS-Bench (topology) api from /work/ws-tmp/g051463-tenneti_NAS/Thesis/data/NATS-tss-v1_0-3ffb9-full/ with fast_mode=True
[2023-01-25 22:35:47] Create NATS-Bench (topology) done with 0/15625 architectures avaliable.

API create done: NATStopology(0/15625 architectures, fast_mode=True, file=)

[2023-01-25 22:35:47] Call the get_more_info function with index=1200, dataset=cifar100, iepoch=None, hp=200, and is_random=True.
[2023-01-25 22:35:47] Call query_index_by_arch with arch=1200
[2023-01-25 22:35:47] Call clear_params with archive_root=/work/ws-tmp/g051463-tenneti_NAS/Thesis/data/NATS-tss-v1_0-3ffb9-full/ and index=1200


/home/g051463/.conda/envs/env_nas/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'test-accuracy': 55.54,
 'test-all-time': 121.94741850807492,
 'test-loss': 1.6072956188201903,
 'test-per-time': 0.6097370925403746,
 'train-accuracy': 61.85,
 'train-all-time': 3415.230631828308,
 'train-loss': 1.3670781739044189,
 'train-per-time': 17.07615315914154,
 'valid-accuracy': 55.39999995727539,
 'valid-all-time': 121.94741850807492,
 'valid-loss': 1.6613771202087402,
 'valid-per-time': 0.6097370925403746,
 'valtest-accuracy': 55.47,
 'valtest-all-time': 243.89483701614984,
 'valtest-loss': 1.6343362113952637,
 'valtest-per-time': 1.2194741850807491}


In [ ]:
from nats_bench import create
# Create the API instance for the size search space in NATS
# api = create(None, 'sss', fast_mode=True, verbose=True)

# Create the API instance for the topology search space in NATS
# api = create('/work/ws-tmp/g051463-tenneti_NAS/NATS-Bench/fake_torch_dir/NATS-tss-v1_0-3ffb9-simple/', 'tss', fast_mode=True, verbose=True)
# api = create('/work/ws-tmp/g051463-tenneti_NAS/data/NATS-tss-v1_0-3ffb9-full/', 'tss', fast_mode=True, verbose=True)
api = create(None, 'tss', fast_mode=True, verbose=True)

In [2]:
config = api.get_net_config(1200, "cifar100")
print(config)

[2023-01-25 22:37:17] Call the get_net_config function with index=1200, dataset=cifar100.
[2023-01-25 22:37:17] Call _prepare_info with index=1200 skip because it is in arch2infos_dict
{'name': 'infer.tiny', 'C': 16, 'N': 5, 'arch_str': '|skip_connect~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|', 'num_classes': 100}


In [3]:
import xautodl
from xautodl.models import get_cell_based_tiny_net
network =  get_cell_based_tiny_net(config)
print(network)

TinyNetwork(
  TinyNetwork(C=16, N=5, L=17)
  (stem): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cells): ModuleList(
    (0): InferCell(
      info :: nodes=4, inC=16, outC=16, [1<-(I0-L0) | 2<-(I0-L1,I1-L2) | 3<-(I0-L3,I1-L4,I2-L5)], |skip_connect~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|
      (layers): ModuleList(
        (0): Identity()
        (1): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): Ba

In [14]:
params = api.get_net_param(1200, 'cifar100',None,hp="200")
print(params)

[2023-01-25 23:01:27] Call the get_net_param function with index=1200, dataset=cifar100, seed=None, hp=200
Call query_meta_info_by_index with arch_index=1200, hp=200
[2023-01-25 23:01:27] Call _prepare_info with index=1200 skip because it is in arch2infos_dict
{777: OrderedDict([('stem.0.weight', tensor([[[[-8.2964e-02, -6.8212e-02, -5.3936e-02],
          [ 8.5075e-02,  1.4554e-01,  1.2761e-01],
          [-1.3669e-01, -3.7396e-02, -1.0313e-01]],

         [[-7.7681e-02, -3.1143e-02, -6.1263e-02],
          [ 1.7869e-01,  2.8117e-01,  1.9188e-01],
          [-1.5274e-01, -1.6082e-02, -1.1131e-01]],

         [[-5.1918e-01, -4.5329e-01, -3.8465e-01],
          [ 1.0595e-01,  3.6098e-01,  3.5560e-01],
          [ 1.1070e-01,  4.4192e-01,  3.5489e-01]]],


        [[[ 1.7412e-01,  4.1579e-01, -2.1199e-01],
          [ 3.7251e-01,  6.1738e-01,  1.0666e-02],
          [-1.4778e-01,  6.4297e-02, -3.4661e-01]],

         [[-1.2322e-01,  1.6360e-01, -2.5000e-01],
          [ 7.6914e-02,  2.61

In [15]:
network.load_state_dict(next(iter(params.values())))


<All keys matched successfully>

2.Query the performance

In [ ]:
# Show the architecture topology string of the 12-th architecture
# For the topology search space, the string is interpreted as
# arch = '|{}~0|+|{}~0|{}~1|+|{}~0|{}~1|{}~2|'.format(
#         edge_node_0_to_node_1,
#         edge_node_0_to_node_2,
#         edge_node_1_to_node_2,
#         edge_node_0_to_node_3,
#         edge_node_1_to_node_3,
#         edge_node_2_to_node_3,
#         )
# For the size search space, the string is interpreted as
# arch = '{}:{}:{}:{}:{}'.format(out_channel_of_1st_conv_layer,
#                                out_channel_of_1st_cell_stage,
#                                out_channel_of_1st_residual_block,
#                                out_channel_of_2nd_cell_stage,
#                                out_channel_of_2nd_residual_block,
#                                )
architecture_str = api.arch(1000)
print(architecture_str)

# Query the loss / accuracy / time for 1234-th candidate architecture on CIFAR-10
# info is a dict, where you can easily figure out the meaning by key
info = api.get_more_info(1000, 'cifar100')
pprint(info)

# Query the flops, params, latency. info is a dict.
info = api.get_cost_info(1000, 'cifar100')
pprint(info)

# Simulate the training of the 1224-th candidate:
validation_accuracy, latency, time_cost, current_total_time_cost = api.simulate_train_eval(1200, dataset='cifar100', hp='200')
sim = [validation_accuracy, latency, time_cost, current_total_time_cost]
pprint(sim)

3.Create Instance of architecture candidate in NATS_Bench

In [9]:
# Create the instance of th 12-th candidate for CIFAR-10.
# To keep NATS-Bench repo concise, we did not include any model-related codes here because they rely on PyTorch.
# The package of [models] is defined at https://github.com/D-X-Y/AutoDL-Projects
#   so that one need to first import this package.
import xautodl
from xautodl.models import get_cell_based_tiny_net
config = api.get_net_config(1200, 'cifar100')
params = api.get_net_param(1200, 'cifar100', None)
network = get_cell_based_tiny_net(config)

# Load the pre-trained weights: params is a dict, where the key is the seed and value is the weights.

network.load_state_dict(next(iter(params.values())))

[2023-01-25 22:59:00] Call the get_net_config function with index=1200, dataset=cifar100.
[2023-01-25 22:59:00] Call _prepare_info with index=1200 skip because it is in arch2infos_dict
[2023-01-25 22:59:00] Call the get_net_param function with index=1200, dataset=cifar100, seed=None, hp=12
Call query_meta_info_by_index with arch_index=1200, hp=12
[2023-01-25 22:59:00] Call _prepare_info with index=1200 skip because it is in arch2infos_dict


<All keys matched successfully>

In [ ]:
pprint(params)

4.Others

In [ ]:
# Clear the parameters of the 12-th candidate.
api.clear_params(1000)

# Reload all information of the 12-th candidate.
api.reload(index=1000)


API Tests

In [ ]:
from nats_bench import api_test
api_test.test_nats_bench_tss('NATS-tss-v1_0-3ffb9-simple')
api_test.test_nats_bench_tss('NATS-sss-v1_0-50262-simple')

In [ ]:
data = api.query_by_index(1200, "cifar100", hp="200")
data[777].train_acc1es[199]


In [ ]:
info = api.get_more_info(1200, 'cifar100', hp="200",is_random=True)
info['train-accuracy']